In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
from google import genai

In [3]:
load_dotenv()

client = genai.Client()

In [4]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [7]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = client.models.generate_content(model="gemini-3-flash-preview", contents=prompt)

    # update state
    state['outline'] = outline.text

    return state

In [11]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a 4-5 line blog on the title - {title} using the follwing outline \n {outline}'

    content = client.models.generate_content(model="gemini-3-flash-preview", contents=prompt)

    state['content'] = content.text

    return state

In [12]:
graph = StateGraph(BlogState)

graph.add_node("create_outline",create_outline)
graph.add_node("create_blog",create_blog)

graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", END)

workflow = graph.compile()

In [13]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': 'This detailed blog outline provides a structured flow to cover the economic, social, and technological aspects of the AI revolution in India.\n\n---\n\n# Blog Title Ideas:\n*   *From IT Hub to AI Powerhouse: The Rise of Artificial Intelligence in India*\n*   *The Silicon Plateau: How India is Shaping the Global AI Landscape*\n*   *AI for All: Inside India’s Unique Path to Artificial Intelligence Mastery*\n\n---\n\n## I. Introduction\n*   **The Hook:** Start with a striking statistic (e.g., India’s potential $957 billion boost to GDP by 2035 via AI).\n*   **The Context:** Briefly trace India’s evolution from an IT outsourcing hub to a global leader in software and now a burgeoning AI ecosystem.\n*   **Thesis Statement:** India is uniquely positioned to lead the AI revolution due to its massive data sets, young talent pool, and government-backed "AI for All" philosophy.\n\n## II. The Government’s Vision: "AI for All"\n*   **NITI Aayog’s Nation